### WebScrapping dos dados

Essa primeira parte consistirá na coleta dos dados com ferramentas de WebScrapping como Selenium e BeautifulSoup, no site de avaliações IMDb pelo link 'https://www.imdb.com/title/tt0388629/episodes/?ref_=tt_eps_sm'.

As informações são relacionadas às avaliações dos episódios do anime One Piece, que se configura como um dos animes mais populares e com maior quantidade de episódios (ainda contando) da atualidade.

#### Bibliotecas utilizadas

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import datetime

#### Link para cada episódio com Selenium

Essa etapa serve para acessar a página principal de avaliações com todos os episódios e pegar o link de cada um para direcionar à uma página com informações exclusivas de cada episódio. 

Não foi possível obter os dados nessa página principal, por isso foi necessário coletar o link externo para cada episódio.

In [4]:
# Inicie o navegador (você precisa ter o driver correspondente, ex: chromedriver)
driver = webdriver.Chrome()

# Acesse a página
url = 'https://www.imdb.com/title/tt0388629/episodes/?ref_=tt_eps_sm'

driver.get(url)

# Aguarde alguns segundos para a página carregar
time.sleep(3)

# Localize e clique no botão "Mostrar tudo" (ajuste o seletor conforme necessário)
# Aguarde até que o botão seja clicável
try:
    button1 = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[2]/article[51]/div/span[2]')))

    # Rolar até o botão antes de clicar
    driver.execute_script("arguments[0].scrollIntoView(true);", button1)
    time.sleep(3)  # Aguarde um segundo para garantir que o botão está visível

    button1.click()  # Clica no botão "Tudo"

    #time.sleep(45)
    time.sleep(10)
    
    episodes = list()

    for i in range(1,1118):

        try:
            episode = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[2]/article[{i}]/div/div/div[3]/div[1]/h4/div/a').get_attribute('href')
            
        except:
            try:
                episode = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[2]/article[{i}]/div/div/div[3]/div[2]/h4/div/a').get_attribute('href')
          
            except:
                print(f'Algo deu errado com o episódio: {i}')

            else:
                # Segunda tentativa foi bem-sucedida
                print(f'Episódio {i} encontrado: {episode}')
        else:
            # Primeira tentativa foi bem-sucedida
            print(f'Episódio {i} encontrado: {episode}')

        episodes.append(episode)            

except Exception as e:
    print(e)

finally: 
    # Feche o navegador
    driver.close()

Episódio 1 encontrado: https://www.imdb.com/title/tt0947442/?ref_=ttep_ep1
Episódio 2 encontrado: https://www.imdb.com/title/tt0947176/?ref_=ttep_ep2
Episódio 3 encontrado: https://www.imdb.com/title/tt0947175/?ref_=ttep_ep3
Episódio 4 encontrado: https://www.imdb.com/title/tt0947174/?ref_=ttep_ep4
Episódio 5 encontrado: https://www.imdb.com/title/tt0948320/?ref_=ttep_ep5
Episódio 6 encontrado: https://www.imdb.com/title/tt0949128/?ref_=ttep_ep6
Episódio 7 encontrado: https://www.imdb.com/title/tt0952145/?ref_=ttep_ep7
Episódio 8 encontrado: https://www.imdb.com/title/tt0952148/?ref_=ttep_ep8
Episódio 9 encontrado: https://www.imdb.com/title/tt0952146/?ref_=ttep_ep9
Episódio 10 encontrado: https://www.imdb.com/title/tt0952147/?ref_=ttep_ep10
Episódio 11 encontrado: https://www.imdb.com/title/tt0957058/?ref_=ttep_ep11
Episódio 12 encontrado: https://www.imdb.com/title/tt0957053/?ref_=ttep_ep12
Episódio 13 encontrado: https://www.imdb.com/title/tt0957059/?ref_=ttep_ep13
Episódio 14 encon

#### Coleta dos dados de cada episódio com Beutiful Soup

Essa etapa consistiu em utilizar o BeautifulSoup para obter as informações de cada link entre os 1117 episódios e gerar um DataFrame final com todos os dados.

In [6]:
episodes_dict = {
    'Temporada': [], # content_data['series']['episodeNumber']['seasonNumber']
    'Episódio': [], # content_data['series']['episodeNumber']['episodeNumber']
    'Título': [], #content_data['titleText']['text']
    'Lançamento': [],
    'Votos': [], #content_data['ratingsSummary']['voteCount']
    'Avaliação': []     #content_data['ratingsSummary']['aggregateRating']
    }

ep_count = 1

for episode in episodes:

    # Simule o comportamento de um navegador adicionando um cabeçalho 'User-Agent'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Faça a requisição usando o cabeçalho
    response = requests.get(episode, headers=headers)

    soup = BeautifulSoup(response.content, 'html.parser')  

    script = soup.find('script', id="__NEXT_DATA__", type="application/json")

    json_data = script.string
                
    # Converta a string JSON em um dicionário Python
    data = json.loads(json_data)

    # Exemplo: Acesse as chaves dentro do dicionário
    content_data = data['props']['pageProps']['aboveTheFoldData']

    episodes_dict['Temporada'].append(content_data['series']['episodeNumber']['seasonNumber'])
    episodes_dict['Episódio'].append(content_data['series']['episodeNumber']['episodeNumber'])
    episodes_dict['Título'].append(content_data['titleText']['text'])

    dia, mês, ano = content_data['releaseDate']['day'], content_data['releaseDate']['month'], content_data['releaseDate']['year']
    episodes_dict['Lançamento'].append(datetime.date(ano, mês, dia))
    
    episodes_dict['Qtd de votos'].append(content_data['ratingsSummary']['voteCount'])
    episodes_dict['Avaliação média'].append(content_data['ratingsSummary']['aggregateRating'])

    print(f'Episódio {ep_count} processado.')

    ep_count += 1

Episódio 1 processado.
Episódio 2 processado.
Episódio 3 processado.
Episódio 4 processado.
Episódio 5 processado.
Episódio 6 processado.
Episódio 7 processado.
Episódio 8 processado.
Episódio 9 processado.
Episódio 10 processado.
Episódio 11 processado.
Episódio 12 processado.
Episódio 13 processado.
Episódio 14 processado.
Episódio 15 processado.
Episódio 16 processado.
Episódio 17 processado.
Episódio 18 processado.
Episódio 19 processado.
Episódio 20 processado.
Episódio 21 processado.
Episódio 22 processado.
Episódio 23 processado.
Episódio 24 processado.
Episódio 25 processado.
Episódio 26 processado.
Episódio 27 processado.
Episódio 28 processado.
Episódio 29 processado.
Episódio 30 processado.
Episódio 31 processado.
Episódio 32 processado.
Episódio 33 processado.
Episódio 34 processado.
Episódio 35 processado.
Episódio 36 processado.
Episódio 37 processado.
Episódio 38 processado.
Episódio 39 processado.
Episódio 40 processado.
Episódio 41 processado.
Episódio 42 processado.
E

In [22]:
df_episodes = pd.DataFrame(episodes_dict)
df_episodes

,Temporada,Episódio,Título,Lançamento,Qtd de votos,Avaliação média
0,1,1,Ore wa Luffy! Kaizoku Ou ni Naru Otoko Da!,1999-10-20,28293,8.4
1,1,2,Daikengo Arawaru! Kaizokugari Roronoa Zoro,1999-11-17,13351,8.3
2,1,3,Morgan vs. Luffy! Nazo no Bishoujo wa Dare?,1999-11-24,5868,8.0
3,1,4,Luffy no Kako! Akagami no Shanks Toujou,1999-12-08,4744,8.3
4,1,5,Kyoufu Nazo no Chikara! Kaizoku Douke Buggy-se...,1999-12-15,3120,7.4
...,...,...,...,...,...,...
1112,1,1113,"Run, Koby! A Desperate Escape Strategy!",2024-07-28,1873,8.3
1113,1,1114,For the Beloved Pupil - The Fist of Vice Admir...,2024-08-04,7681,9.7
1114,1,1115,The Navy Surprised! The Navy Headquarters' For...,2024-08-11,3891,9.7
1115,1,1116,Let's Go Get It! Buggy's Big Declaration,2024-08-18,1196,8.3


In [24]:
df_episodes.to_excel('C:/Users/USER/Documents/EstudosDados/Projetos/One Piece/episodes.xlsx', index=0)